In [2]:
import pandas as pd
import numpy as np
import calendar

In [39]:
data = pd.read_pickle('data/merged_with_missing.pickle')
data.head()

,session_id,time,user_id,product_id,event,discount,purchase_id,made_purchase,known_product,is_user_invalid,...,duration,syntetic_duration,agg_time_spent,weekday,hour,weekday_sin,weekday_cos,hour_sin,hour_cos,ends_with_purchase
0,100001.0,2020-01-17 16:08:57,102.0,1001.0,view_product,0,NaN,False,True,False,...,00:02:32.306757,True,00:07:44.613515,4,16,-0.433884,-0.900969,-0.866025,-0.500000,False
1,100002.0,2020-01-07 05:10:41,102.0,1277.0,view_product,20,NaN,False,True,False,...,00:03:45,False,00:05:12.306757,1,5,0.781831,0.623490,0.965926,0.258819,False
2,100002.0,2020-01-07 05:14:26,102.0,1276.0,view_product,20,NaN,False,True,False,...,00:02:32.306757,True,00:03:45,1,5,0.781831,0.623490,0.965926,0.258819,False
3,100003.0,2020-01-16 09:37:42,102.0,1276.0,view_product,0,NaN,False,True,False,...,00:00:40,False,00:06:17.306757,3,9,0.433884,-0.900969,0.707107,-0.707107,False
4,100003.0,2020-01-16 09:38:22,102.0,1277.0,view_product,0,NaN,False,True,False,...,00:02:32.306757,True,00:08:57.306757,3,9,0.433884,-0.900969,0.707107,-0.707107,False


# Missing data analysis

To decide if missing data is MCAR or MAR problem we are using mutual_info_classif from sklearn library to estimate mutual information between each feature and the target ('is_user_invalid', 'is_delivery_invalid', 'is_price_invalid', 'known_product').

Mutual information (MI) between two random variables is a non-negative value, which measures the dependency between the variables. It is equal to zero if and only if two random variables are independent, and higher values mean higher dependency.

In [66]:
from sklearn.feature_selection import mutual_info_classif
np.set_printoptions(suppress=True)

target_columns_with_input = {
    # 'y_col': ['x_col1', 'x_col2' ... ]
    'is_price_invalid': ['discount', ],
    'is_delivery_invalid': ['discount', ]
}
discrete_x_cols = []
y_columns = ['is_user_invalid', 'is_delivery_invalid', 'is_price_invalid', 'known_product']

In [72]:
for target_col in target_columns_with_input.keys():
    x_cols = target_columns_with_input[target_col]
    print(f'\n\nFor target column {target_col} we are analysing features {x_cols}')
    mi = mutual_info_classif(
        data[x_cols],
        data[target_col].astype('bool'), 
        copy=True,
        #discrete_features=[c in discrete_x_cols for c in x_columns]
    )
    print('received MI scores', mi)



For target column is_price_invalid we are analysing features ['discount']
received MI scores [0.00183428]


For target column is_delivery_invalid we are analysing features ['discount']
received MI scores [0.00026448]


# Conclusions